In [1]:
# alc_pretrain_path = "/home/guangda/repos/LabelMaker-Pointcept/exp/alc/arkitscenes_labelmaker_scannet200_pretrai_regular_lr/model/model_best.pth"
alc_pretrain_path = "/home/guangda/repos/LabelMaker-Pointcept/exp/alc/arkitscenes_labelmaker_wn199_pretrain/model/model_last.pth"

In [2]:
import torch

a = torch.load(alc_pretrain_path)

import os
import sys

sys.path.append("/home/guangda/repos/LabelMaker-Pointcept")

In [3]:
from pointcept.models import  PointTransformerV3
from pointcept.models.default import DefaultSegmentorV2
from pointcept.models.losses import CrossEntropyLoss, LovaszLoss

In [4]:
empty_model = DefaultSegmentorV2(
    # num_classes=20,
    # num_classes=200,
    num_classes=100,
    backbone_out_channels=64,
    backbone=dict(
        type="PT-v3m1",
        in_channels=6,
        order=["z", "z-trans", "hilbert", "hilbert-trans"],
        stride=(2, 2, 2, 2),
        enc_depths=(2, 2, 2, 6, 2),
        enc_channels=(32, 64, 128, 256, 512),
        enc_num_head=(2, 4, 8, 16, 32),
        enc_patch_size=(1024, 1024, 1024, 1024, 1024),
        dec_depths=(2, 2, 2, 2),
        dec_channels=(64, 64, 128, 256),
        dec_num_head=(4, 4, 8, 16),
        dec_patch_size=(1024, 1024, 1024, 1024),
        mlp_ratio=4,
        qkv_bias=True,
        qk_scale=None,
        attn_drop=0.0,
        proj_drop=0.0,
        drop_path=0.3,
        shuffle_orders=True,
        pre_norm=True,
        enable_rpe=False,
        enable_flash=True,
        upcast_attention=False,
        upcast_softmax=False,
        cls_mode=False,
        pdnorm_bn=False,
        pdnorm_ln=False,
        pdnorm_decouple=True,
        pdnorm_adaptive=False,
        pdnorm_affine=True,
        pdnorm_conditions=("ScanNet", "S3DIS", "Structured3D"),
    ),
    criteria=[
        dict(type="CrossEntropyLoss", loss_weight=1.0, ignore_index=-1),
        dict(type="LovaszLoss", mode="multiclass", loss_weight=1.0, ignore_index=-1),
    ],
)

In [5]:
old_keys = set([key.replace("module.", "") for key in a["state_dict"].keys()])
new_keys = set(empty_model.state_dict().keys())

In [6]:
old_keys - new_keys

set()

In [7]:
new_keys - old_keys

set()

In [8]:
for key in new_keys:
    old_key = "module." + key
    if a["state_dict"][old_key].shape != empty_model.state_dict()[key].shape:
        print(key)

seg_head.bias
seg_head.weight


In [9]:
new_save = {"state_dict": {}}
for key in new_keys:
    old_key = "module." + key
    if a["state_dict"][old_key].shape != empty_model.state_dict()[key].shape:
        new_save["state_dict"][old_key] = empty_model.state_dict()[key]
    else:
        new_save["state_dict"][old_key] = a["state_dict"][old_key]

In [10]:
# torch.save(new_save, "/home/guangda/repos/LabelMaker-Pointcept/exp/alc/arkitscenes_labelmaker_wn199_pretrain/model/model_modified_scannet200.pth")
torch.save(new_save, "/home/guangda/repos/LabelMaker-Pointcept/exp/alc/arkitscenes_labelmaker_wn199_pretrain/model/model_modified_scannetpp.pth")